# Implementation of Spiral Path Algorithm
1. Define a boundary radius around the region centre
2. Establish how many rings around the centre to be searched
3. Create a way to search the radius algorithmically
4. Return the path

In [29]:
import h3
import folium
from math import sqrt

## Visualisation Functions

In [30]:
# Return a colour hex value from a value between 0-1
def gradient_color(value):
    if value < 0:
        value = 0
    elif value > 1:
        value = 1

    r = int(255 * value)
    b = int(255 * value)
    g = int(255 * value)

    hex_string = '#{:02x}{:02x}{:02x}'.format(r, b, g)
    return hex_string

In [31]:
# Add H3 hexagons as polygons to the map, m
def add_hex_to_map(hexagon_values, m):
    keys = list(hexagon_values.keys())
    num_keys = len(keys)

    for i, hexagon_id in enumerate(hexagon_values):
        vertices = h3.h3_to_geo_boundary(hexagon_id)
        color = color = gradient_color(hexagon_values[hexagon_id])
        folium.Polygon(locations=vertices, color=color, fill=True, fill_opacity=0.6).add_to(m)
        folium.map.Marker(h3.h3_to_geo(hexagon_id),
                icon=folium.DivIcon(
                    icon_size=(10,10),
                    icon_anchor=(5,14),
                    html=f'<div style="font-size: 8pt">{i}</div>'
                )
                ).add_to(m)

In [32]:
def norm_dict_from_ls(input_list):
    if not input_list:
        return {}

    list_length = len(input_list)
    result_dict = {}

    for index, value in enumerate(input_list):
        normalized_index = index / list_length
        result_dict[value] = normalized_index

    return result_dict

## Path Algo Implementation

In [204]:
# SUTD Campus
centre = (1.340640225800291, 103.962424829341)
res = 12
search_radius_m = 300
# res = 12
# search_radius_m = 200


In [205]:
anchor_hex = h3.geo_to_h3(centre[0], centre[1],res)

In [206]:
hex_edge_length = h3.edge_length(resolution=res, unit='m')
hex_apothem = hex_edge_length * sqrt(3) / 2
hexagon_area = 1/2 * hex_edge_length * 6 * hex_apothem

In [207]:
# Number of rings to search around the central hexagon
k_ring_radius = (search_radius_m-hex_apothem)/hex_apothem/2
k_ring_radius

17.89568822356688

In [208]:
center_hexagon = h3.geo_to_h3(centre[0],centre[1],resolution=res)
center_ij_coord = h3.experimental_h3_to_local_ij(center_hexagon,center_hexagon)
center_hexagon, center_ij_coord

('8c6526ac34217ff', (70858, 21263))

In [209]:
def naive_path_generator(center_ij_coord, k_ring_radius):

    def ring_edge_traversal(repetitions, current_ij_coord, i_increment, j_increment):
        for i in range(repetitions):
            current_ij_coord = (current_ij_coord[0]+i_increment, current_ij_coord[1]+j_increment)
            path.append(current_ij_coord)
        return current_ij_coord

    path = [(center_ij_coord[0],center_ij_coord[1])]
    current_ij_coord = center_ij_coord
    for ring_r in range(1, k_ring_radius+1):

        current_ij_coord = ring_edge_traversal(1, current_ij_coord, 0, -1)
        current_ij_coord = ring_edge_traversal(ring_r-1, current_ij_coord, 1, 0)
        current_ij_coord = ring_edge_traversal(ring_r, current_ij_coord, 1, 1)
        current_ij_coord = ring_edge_traversal(ring_r, current_ij_coord, 0, 1)
        current_ij_coord = ring_edge_traversal(ring_r, current_ij_coord, -1, 0)
        current_ij_coord = ring_edge_traversal(ring_r, current_ij_coord, -1, -1)
        current_ij_coord = ring_edge_traversal(ring_r, current_ij_coord, 0, -1)

    return path

## Visualisation

In [210]:
m = folium.Map(location=[centre[0],centre[1]], zoom_start=23)

In [211]:
ij_path = naive_path_generator(center_ij_coord, int(k_ring_radius))

In [212]:
hex_path = [h3.experimental_local_ij_to_h3(anchor_hex, x[0], x[1]) for x in ij_path]

In [213]:
hex_path

['8c6526ac34217ff',
 '8c6526ac34213ff',
 '8c6526ac34211ff',
 '8c6526ac34215ff',
 '8c6526ac3423bff',
 '8c6526ac342edff',
 '8c6526ac342e9ff',
 '8c6526ac342c5ff',
 '8c6526ac342cdff',
 '8c6526ac3421bff',
 '8c6526ac34219ff',
 '8c6526ac3421dff',
 '8c6526ac34203ff',
 '8c6526ac34239ff',
 '8c6526ac34231ff',
 '8c6526ac34233ff',
 '8c6526ac342e5ff',
 '8c6526ac342e1ff',
 '8c6526ac342ebff',
 '8c6526ac342c7ff',
 '8c6526ac342c1ff',
 '8c6526ac342c9ff',
 '8c6526ac35527ff',
 '8c6526ac35525ff',
 '8c6526ac34253ff',
 '8c6526ac34257ff',
 '8c6526ac3420bff',
 '8c6526ac34201ff',
 '8c6526ac34207ff',
 '8c6526ac3423dff',
 '8c6526ac34235ff',
 '8c6526ac34237ff',
 '8c6526ac34059ff',
 '8c6526ac3405bff',
 '8c6526ac342e7ff',
 '8c6526ac342e3ff',
 '8c6526ac3428dff',
 '8c6526ac34289ff',
 '8c6526ac342c3ff',
 '8c6526ac342cbff',
 '8c6526ac35535ff',
 '8c6526ac35523ff',
 '8c6526ac35521ff',
 '8c6526ac3552dff',
 '8c6526ac3425bff',
 '8c6526ac34251ff',
 '8c6526ac34255ff',
 '8c6526ac34209ff',
 '8c6526ac3420dff',
 '8c6526ac34205ff',


In [214]:
hex_line_d = norm_dict_from_ls(hex_path)
print(hex_line_d)

{'8c6526ac34217ff': 0.0, '8c6526ac34213ff': 0.001088139281828074, '8c6526ac34211ff': 0.002176278563656148, '8c6526ac34215ff': 0.003264417845484222, '8c6526ac3423bff': 0.004352557127312296, '8c6526ac342edff': 0.00544069640914037, '8c6526ac342e9ff': 0.006528835690968444, '8c6526ac342c5ff': 0.007616974972796518, '8c6526ac342cdff': 0.008705114254624592, '8c6526ac3421bff': 0.009793253536452665, '8c6526ac34219ff': 0.01088139281828074, '8c6526ac3421dff': 0.011969532100108813, '8c6526ac34203ff': 0.013057671381936888, '8c6526ac34239ff': 0.014145810663764961, '8c6526ac34231ff': 0.015233949945593036, '8c6526ac34233ff': 0.01632208922742111, '8c6526ac342e5ff': 0.017410228509249184, '8c6526ac342e1ff': 0.018498367791077257, '8c6526ac342ebff': 0.01958650707290533, '8c6526ac342c7ff': 0.020674646354733407, '8c6526ac342c1ff': 0.02176278563656148, '8c6526ac342c9ff': 0.022850924918389554, '8c6526ac35527ff': 0.023939064200217627, '8c6526ac35525ff': 0.025027203482045703, '8c6526ac34253ff': 0.0261153427638737

In [215]:
add_hex_to_map(hex_line_d, m)
display(m)

In [191]:
m.save('../output/spiral_path.html')